In [1]:
import sys

import torch
import numpy as np

import attacks
from basemodels import TorchModelContainer, IrisNN, BCNN, MnistCnnCW
from datasets import DATASET_LIST, DataContainer, get_dataset_list
import utils
# from defences import DefenceContainer

%load_ext autoreload
%autoreload 2

In [2]:
a = np.random.rand(3, 38, 38, 1)
r = utils.get_image_range(a)
r

(0.00037869555468417193, 0.9997808588429377)

In [3]:
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [4]:
print('Avaliable datasets:')
print(get_dataset_list())

Avaliable datasets:
['MNIST', 'CIFAR10', 'SVHN', 'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed']


In [5]:
DATA_ROOT = 'data'
BATCH_SIZE = 128

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'MNIST'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
# model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
# model = BCNN(num_features, num_classes)
model = MnistCnnCW()
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = TorchModelContainer(model, dc)
mc.fit(epochs=3, batch_size=BATCH_SIZE)

Starting MNIST data container...
{'name': 'MNIST', 'type': 'image', 'size': 70000, 'num_classes': 10, 'dim_data': (1, 28, 28)}
Loading data...
Preparing DataLoaders...
Preparing numpy arrays...
Successfully load data! Time taken:  0m 4.0s
Features: 1
Classes: 10
Using model: MnistCnnCW
Using device: cuda:0
{'lr': 0.1, 'momentum': 0}
[ 1/3]  0m 2.4s - Train Loss: 1.4005 Acc: 50.6433% - Test Loss: 0.1329 Acc: 95.7300%
[ 2/3]  0m 2.4s - Train Loss: 0.1032 Acc: 96.7817% - Test Loss: 0.0608 Acc: 98.0900%
[ 3/3]  0m 2.4s - Train Loss: 0.0621 Acc: 98.0383% - Test Loss: 0.0464 Acc: 98.5100%
Time taken for training:  0m 7.2s


In [6]:
dc.data_test_np.shape

(10000, 28, 28, 1)

In [7]:
attack = attacks.JacobianSaliencyContainer(mc)
print(attack.attack_params)

(0.0, 1.0)
{'theta': 0.1, 'gamma': 1.0, 'batch_size': 8}


In [8]:
n = 32
targets = attack.randam_targets(n, dc.num_classes, use_onehot=True)
adv, y_adv, x_clean, y_clean = attack.generate(count=n, targets=targets)
# adv, y_adv, x_clean, y_clean = attack.generate(count=n)
print(attack.attack_params)
print(adv.shape)
print(y_adv.shape)
print(x_clean.shape)
print(y_clean.shape)

Time taken for training 32 adversarial examples:  0m 4.4s
{'theta': 0.1, 'gamma': 1.0, 'batch_size': 8}
(32, 1, 28, 28)
(32,)
(32, 1, 28, 28)
(32,)


In [12]:
accuracy = mc.evaluate(x_clean, y_clean)
print(f'Accuracy on clean samples: {accuracy*100:.4f}%')
accuracy = mc.evaluate(adv, y_clean)
print(f'Accuracy on adversarial example: {accuracy*100:.4f}%')

Accuracy on clean samples: 100.0000%
Accuracy on adversarial example: 6.2500%
